### Naukri Web Scraper

Ajay Kasturirangan
09-July-2022

#### Import Required Libraries

In [1]:
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
import math

#### Input

In [2]:
Job = "Data Analyst"
Location = "Chennai"

#### Identify Data Source

In [3]:
#url = 'https://www.naukri.com/' + Job.replace(" ","-") + '-jobs-in-' + Location + '?k=' + Job.replace(" ","%20") + '&l=bangalore'
#url
url = 'https://www.naukri.com/'+ Job.replace(" ","-") + '-jobs-in-' + Location
url

'https://www.naukri.com/Data-Analyst-jobs-in-Chennai'

#### Importing Data

In [4]:
def import_data(url):
    page = requests.get(url)
    driver = webdriver.Chrome('E:\\chromedriver_win32 (1)\\chromedriver.exe')
    driver.get(url)
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source,'html5lib')
    driver.close()
    return soup

#### Page Counter

In [5]:
def page_counter(soup):
    results = soup.find(class_="sortAndH1Cont")
    display = results.find_all('span',class_="fleft grey-text mr-5 fs12")
    return math.ceil(int(display[0].text.split("of ")[1])/20)

#### Extract Info

In [6]:
def extract_info(postings,soup):
    results = soup.find(class_='list')
    jobs = results.find_all('article',class_='jobTuple bgWhite br4 mb-8')
    for job in jobs:
        URL = job.find('a',class_ = "title fw500 ellipsis").get('href')
        title = job.find('a',class_ = "title fw500 ellipsis")
        exp = job.find('span',class_ ="ellipsis fleft fs12 lh16").text
        company = job.find('a',class_="subTitle ellipsis fleft").text
        SKILLS =  job.find_all('li',class_="fleft fs12 grey-text lh16 dot")
        skills = []
        for skill in SKILLS:
            skills.append(skill.text)
        
        salaries = job.find_all('li',class_="fleft grey-text br2 placeHolderLi salary")
        amounts = []
        for salary in salaries:
            amount = salary.find('span',class_ = 'ellipsis fleft fs12 lh16').text
            if amount.upper() == "NOT DISCLOSED":
                amounts = [amount]
            else:
                amounts = [int(i) for i in amount.replace("PA","").replace(",","").replace(".","").split("-")]
    
        postings = postings.append({'URL':URL,'Title':title,'Exp':exp,'Company':company,'Skills':skills,'Salary':amounts},ignore_index=True)    
                
        
    return postings   

#### Main

In [7]:
%%time
jobsearch = pd.DataFrame(columns=['URL','Title','Exp','Company','Skills','Salary'])
soup1 = import_data(url)
pages = page_counter(soup1)
print('Total number of pages for {} in {} is {}'.format(Job,Location,pages))

<ipython-input-4-7c7641d5e5bf>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('E:\\chromedriver_win32 (1)\\chromedriver.exe')


Total number of pages for Data Analyst in Chennai is 84
Wall time: 44.2 s


Scraping only 10 pages for faster implementation

In [8]:
%%time
pages = 10
for i in range(1,pages+1):
    page_url = url+"-"+ str(i)
    page_soup = import_data(page_url)
    jobsearch = extract_info(jobsearch,page_soup)
jobsearch.to_csv('jobsearch.csv')

<ipython-input-4-7c7641d5e5bf>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('E:\\chromedriver_win32 (1)\\chromedriver.exe')


Wall time: 7min 31s


In [9]:
jobsearch.head()

,URL,Title,Exp,Company,Skills,Salary
0,https://www.naukri.com/job-listings-data-analy...,[Data Analyst - CRM Platform],1-6 Yrs,Artech infosystem,"[MDM, Analytics, CRM, Java, communication, Sal...",[Not disclosed]
1,https://www.naukri.com/job-listings-sr-associa...,[Sr. Associate - Data Analyst],1-2 Yrs,Sutherland,"[Advance excel, Data Management, Power Bi, VLO...",[Not disclosed]
2,https://www.naukri.com/job-listings-hiring-dat...,[Hiring Data Analysts - Virtusa],4-8 Yrs,Virtusa,"[Pytorch, AI, PowerBI, statistical analysis, S...",[Not disclosed]
3,https://www.naukri.com/job-listings-data-analy...,[Data Analyst - Python/Artificial Intelligence],0-2 Yrs,iMindYourBusiness,"[Data Science, STEM, word, SQL server, VBA, Ad...",[Not disclosed]
4,https://www.naukri.com/job-listings-business-a...,[Business Analyst / Data Analyst],5-10 Yrs,cliqhr.com,"[data analysis, data quality, Business Analyst...",[Not disclosed]


In [10]:
jobsearch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   URL      199 non-null    object
 1   Title    199 non-null    object
 2   Exp      199 non-null    object
 3   Company  199 non-null    object
 4   Skills   199 non-null    object
 5   Salary   199 non-null    object
dtypes: object(6)
memory usage: 9.5+ KB


In [11]:
jobsearch.to_csv('jobsearch.csv') 